In [1]:
import cv2
import numpy as np
import time
import imutils
from joblib import load
from dataset_io import *
import timeit
from tqdm import tqdm
import psutil
from datetime import datetime

In [2]:
def pyramid(image, split_channels, initial_scale, final_scale, scale=1.5):
    original_w=image.shape[1]
    curr_scale=initial_scale
    while curr_scale>final_scale/scale:
#         print(curr_scale,final_scale, 24/curr_scale, 24/final_scale)
        w = int(original_w * curr_scale)
        yield imutils.resize(image, width=w), [imutils.resize(ch, width=w) for ch in split_channels],curr_scale
        curr_scale/=scale
        
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [3]:
(winW, winH) = (120,24)
DS_min_winH_m=41
DS_max_winH_m=287
initial_scale=winH/DS_min_winH_m
final_scale=winH/DS_max_winH_m

clf = load('rand_forest_clf.joblib') 
clf.set_params(n_jobs=-1)

def show(img, x, y, winW, winH, positives=[]):
    clone = img.copy()
    cv2.rectangle(clone, (x, y), (x + winW, y + winH), (255, 255, 255), 2)
    for p in positives:
        x,y=p
        cv2.rectangle(clone, (x, y), (x + winW, y + winH), (255, 255, 255), 2)
    cv2.imshow("Window", clone)
    cv2.waitKey(1)

def global_predict():
    global glob_X, glob_Y
    if len(glob_X)!=0:
        pred=clf.predict(glob_X)
        if glob_Y.size == 0:
            glob_Y=pred
        else:
            glob_Y=np.concatenate((glob_Y,pred))
        glob_X=[]
    
def predict(filename, scene_name, label_resolution, rects):
    global glob_X,glob_IDX,glob_IT,glob_DUMP_IT
    
    image=imread_resized(scene_name, filename, label_resolution)
    split_channels=read_split_channels(scene_name, filename)
    X=[]
    for resized_img, resized_split_ch, scale in pyramid(image, split_channels, initial_scale, final_scale, scale=1.15):
#         draw_preds=[]
        resized_ch=np.dstack(resized_split_ch)
        for (x, y, window) in sliding_window(resized_img, stepSize=10, windowSize=(winW, winH)):
            # if the window does not meet our desired window size, ignore it
            if window.shape[0] != winH or window.shape[1] != winW:
                continue
            channels_window=resized_ch[y:y + winH, x:x + winW]
            features=channels_window.flatten()
            glob_X.append(features)
            glob_IDX.append([scene_name, filename, scale, (x,y)])
#             if clf.predict([features]):
#                 draw_preds.append((x, y))
            #show(resized_img, x, y, winW, winH, draw_preds)
    glob_IT+=1
    if glob_IT==glob_DUMP_IT:
        global_predict()
        glob_IT=0

In [4]:
glob_X=[]
glob_IDX=[]
glob_Y=np.array([])
glob_IT=0
glob_DUMP_IT=1 # adjust manually to RAM

[predict(*row) for row in tqdm([row for row in walk_dataset() if row[1]=="blue_1"])]# and row[0]=="IMG_20200229_075416.jpg"])]
global_predict()

now = datetime.now()
date_time = now.strftime("%d-%m-%Y_%H:%M:%S")
np.save("run/sliding_window_IDX_"+date_time,glob_IDX)
np.save("run/sliding_window_Y_"+date_time,glob_Y)
print(len(glob_IDX), len(glob_Y),len(glob_X))

#glob_IDX=np.load("run/sliding_window_IDX_07-04-2020_17:59:14.npy",allow_pickle=True)
#glob_Y=np.load("run/sliding_window_Y_07-04-2020_17:59:14.npy",allow_pickle=True)
glob_Y=np.array([[e] for e in glob_Y])
RES=np.append(glob_IDX, glob_Y, 1)
print(np.array(glob_IDX).shape, glob_Y.shape, RES.shape)

100%|██████████| 48/48 [00:26<00:00,  1.80it/s]


349872 349872 0
(349872, 4) (349872, 1) (349872, 5)


In [5]:
print(RES)

[['blue_1' 'IMG_20200229_075347.jpg' 0.5853658536585366 (0, 0) 0.0]
 ['blue_1' 'IMG_20200229_075347.jpg' 0.5853658536585366 (10, 0) 0.0]
 ['blue_1' 'IMG_20200229_075347.jpg' 0.5853658536585366 (20, 0) 0.0]
 ...
 ['blue_1' 'IMG_20200229_075507.jpg' 0.12582042305248736 (10, 60) 0.0]
 ['blue_1' 'IMG_20200229_075507.jpg' 0.12582042305248736 (0, 70) 0.0]
 ['blue_1' 'IMG_20200229_075507.jpg' 0.12582042305248736 (10, 70) 0.0]]


In [6]:
def scale_many(vals, scale):
    return [int(val/scale) for val in vals]

def draw_predicted_rectangles(filename, scene_name, label_resolution, rects):
    img=imread_resized(scene_name, filename, label_resolution)
    file_res=RES[np.where((RES[:,0] == scene_name) * (RES[:,1] == filename))]
    for row in file_res:
        [scene_name, filename, scale, (x,y), pred]=row
        [x,y,winW_s,winH_s]=scale_many([x,y,winW,winH], scale)
        #print(winH_s)
        if pred:
            cv2.rectangle(img, (x,y), (x + winW_s, y + winH_s), (0, 255, 0), 2)
            print(row)
    save_image(img, scene_name, filename, "predicted_labels")

[draw_predicted_rectangles(*row) for row in walk_dataset() if row[1]=="blue_1"]# and row[0]=="IMG_20200229_075416.jpg"]
print("done")

['blue_1' 'IMG_20200229_075347.jpg' 0.5853658536585366 (240, 180) 1.0]
['blue_1' 'IMG_20200229_075347.jpg' 0.5853658536585366 (20, 200) 1.0]
['blue_1' 'IMG_20200229_075347.jpg' 0.5853658536585366 (30, 200) 1.0]
['blue_1' 'IMG_20200229_075347.jpg' 0.5853658536585366 (40, 200) 1.0]
['blue_1' 'IMG_20200229_075347.jpg' 0.5853658536585366 (50, 200) 1.0]
['blue_1' 'IMG_20200229_075347.jpg' 0.5853658536585366 (60, 200) 1.0]
['blue_1' 'IMG_20200229_075347.jpg' 0.5853658536585366 (70, 200) 1.0]
['blue_1' 'IMG_20200229_075347.jpg' 0.5853658536585366 (80, 200) 1.0]
['blue_1' 'IMG_20200229_075347.jpg' 0.5853658536585366 (90, 200) 1.0]
['blue_1' 'IMG_20200229_075347.jpg' 0.5090137857900319 (190, 130) 1.0]
['blue_1' 'IMG_20200229_075347.jpg' 0.5090137857900319 (280, 130) 1.0]
['blue_1' 'IMG_20200229_075347.jpg' 0.5090137857900319 (330, 130) 1.0]
['blue_1' 'IMG_20200229_075347.jpg' 0.5090137857900319 (180, 150) 1.0]
['blue_1' 'IMG_20200229_075347.jpg' 0.5090137857900319 (190, 150) 1.0]
['blue_1' 'IMG

['blue_1' 'IMG_20200229_075349.jpg' 0.5853658536585366 (410, 170) 1.0]
['blue_1' 'IMG_20200229_075349.jpg' 0.5853658536585366 (40, 180) 1.0]
['blue_1' 'IMG_20200229_075349.jpg' 0.5853658536585366 (50, 180) 1.0]
['blue_1' 'IMG_20200229_075349.jpg' 0.5853658536585366 (60, 180) 1.0]
['blue_1' 'IMG_20200229_075349.jpg' 0.5853658536585366 (70, 180) 1.0]
['blue_1' 'IMG_20200229_075349.jpg' 0.5853658536585366 (80, 180) 1.0]
['blue_1' 'IMG_20200229_075349.jpg' 0.5853658536585366 (100, 180) 1.0]
['blue_1' 'IMG_20200229_075349.jpg' 0.5853658536585366 (110, 180) 1.0]
['blue_1' 'IMG_20200229_075349.jpg' 0.5853658536585366 (120, 180) 1.0]
['blue_1' 'IMG_20200229_075349.jpg' 0.5853658536585366 (130, 180) 1.0]
['blue_1' 'IMG_20200229_075349.jpg' 0.5853658536585366 (140, 180) 1.0]
['blue_1' 'IMG_20200229_075349.jpg' 0.5853658536585366 (150, 180) 1.0]
['blue_1' 'IMG_20200229_075349.jpg' 0.5853658536585366 (160, 180) 1.0]
['blue_1' 'IMG_20200229_075349.jpg' 0.5853658536585366 (170, 180) 1.0]
['blue_1' '

['blue_1' 'IMG_20200229_075350.jpg' 0.5853658536585366 (360, 170) 1.0]
['blue_1' 'IMG_20200229_075350.jpg' 0.5853658536585366 (60, 180) 1.0]
['blue_1' 'IMG_20200229_075350.jpg' 0.5853658536585366 (70, 180) 1.0]
['blue_1' 'IMG_20200229_075350.jpg' 0.5853658536585366 (120, 180) 1.0]
['blue_1' 'IMG_20200229_075350.jpg' 0.5853658536585366 (430, 190) 1.0]
['blue_1' 'IMG_20200229_075350.jpg' 0.5853658536585366 (440, 190) 1.0]
['blue_1' 'IMG_20200229_075350.jpg' 0.5853658536585366 (450, 190) 1.0]
['blue_1' 'IMG_20200229_075350.jpg' 0.5853658536585366 (460, 190) 1.0]
['blue_1' 'IMG_20200229_075350.jpg' 0.5853658536585366 (470, 190) 1.0]
['blue_1' 'IMG_20200229_075350.jpg' 0.5853658536585366 (480, 190) 1.0]
['blue_1' 'IMG_20200229_075350.jpg' 0.5853658536585366 (30, 200) 1.0]
['blue_1' 'IMG_20200229_075350.jpg' 0.5853658536585366 (40, 200) 1.0]
['blue_1' 'IMG_20200229_075350.jpg' 0.5853658536585366 (50, 200) 1.0]
['blue_1' 'IMG_20200229_075350.jpg' 0.5853658536585366 (60, 200) 1.0]
['blue_1' 'I

['blue_1' 'IMG_20200229_075353.jpg' 0.5853658536585366 (440, 120) 1.0]
['blue_1' 'IMG_20200229_075353.jpg' 0.5853658536585366 (200, 220) 1.0]
['blue_1' 'IMG_20200229_075353.jpg' 0.5853658536585366 (210, 220) 1.0]
['blue_1' 'IMG_20200229_075353.jpg' 0.5853658536585366 (220, 220) 1.0]
['blue_1' 'IMG_20200229_075353.jpg' 0.5853658536585366 (370, 220) 1.0]
['blue_1' 'IMG_20200229_075353.jpg' 0.5853658536585366 (380, 220) 1.0]
['blue_1' 'IMG_20200229_075353.jpg' 0.5853658536585366 (390, 220) 1.0]
['blue_1' 'IMG_20200229_075353.jpg' 0.5853658536585366 (400, 220) 1.0]
['blue_1' 'IMG_20200229_075353.jpg' 0.5853658536585366 (160, 250) 1.0]
['blue_1' 'IMG_20200229_075353.jpg' 0.5853658536585366 (220, 250) 1.0]
['blue_1' 'IMG_20200229_075353.jpg' 0.5853658536585366 (230, 250) 1.0]
['blue_1' 'IMG_20200229_075353.jpg' 0.5853658536585366 (280, 250) 1.0]
['blue_1' 'IMG_20200229_075353.jpg' 0.5853658536585366 (290, 250) 1.0]
['blue_1' 'IMG_20200229_075353.jpg' 0.5853658536585366 (310, 250) 1.0]
['blue

['blue_1' 'IMG_20200229_075358.jpg' 0.5853658536585366 (150, 170) 1.0]
['blue_1' 'IMG_20200229_075358.jpg' 0.5853658536585366 (160, 170) 1.0]
['blue_1' 'IMG_20200229_075358.jpg' 0.5853658536585366 (170, 170) 1.0]
['blue_1' 'IMG_20200229_075358.jpg' 0.5853658536585366 (180, 170) 1.0]
['blue_1' 'IMG_20200229_075358.jpg' 0.5853658536585366 (190, 170) 1.0]
['blue_1' 'IMG_20200229_075358.jpg' 0.5853658536585366 (200, 170) 1.0]
['blue_1' 'IMG_20200229_075358.jpg' 0.5853658536585366 (210, 170) 1.0]
['blue_1' 'IMG_20200229_075358.jpg' 0.5853658536585366 (220, 170) 1.0]
['blue_1' 'IMG_20200229_075358.jpg' 0.5853658536585366 (230, 170) 1.0]
['blue_1' 'IMG_20200229_075358.jpg' 0.5853658536585366 (240, 170) 1.0]
['blue_1' 'IMG_20200229_075358.jpg' 0.5853658536585366 (250, 170) 1.0]
['blue_1' 'IMG_20200229_075358.jpg' 0.5853658536585366 (260, 170) 1.0]
['blue_1' 'IMG_20200229_075358.jpg' 0.5853658536585366 (270, 170) 1.0]
['blue_1' 'IMG_20200229_075358.jpg' 0.5853658536585366 (310, 170) 1.0]
['blue

...

['blue_1' 'IMG_20200229_075500.jpg' 0.5853658536585366 (210, 190) 1.0]
['blue_1' 'IMG_20200229_075500.jpg' 0.5853658536585366 (220, 190) 1.0]
['blue_1' 'IMG_20200229_075500.jpg' 0.5853658536585366 (280, 190) 1.0]
['blue_1' 'IMG_20200229_075500.jpg' 0.5853658536585366 (340, 190) 1.0]
['blue_1' 'IMG_20200229_075500.jpg' 0.5853658536585366 (350, 190) 1.0]
['blue_1' 'IMG_20200229_075500.jpg' 0.5853658536585366 (360, 190) 1.0]
['blue_1' 'IMG_20200229_075500.jpg' 0.5853658536585366 (370, 190) 1.0]
['blue_1' 'IMG_20200229_075500.jpg' 0.5853658536585366 (380, 190) 1.0]
['blue_1' 'IMG_20200229_075500.jpg' 0.5853658536585366 (390, 190) 1.0]
['blue_1' 'IMG_20200229_075500.jpg' 0.5853658536585366 (400, 190) 1.0]
['blue_1' 'IMG_20200229_075500.jpg' 0.5853658536585366 (380, 200) 1.0]
['blue_1' 'IMG_20200229_075500.jpg' 0.5853658536585366 (390, 200) 1.0]
['blue_1' 'IMG_20200229_075500.jpg' 0.5853658536585366 (380, 210) 1.0]
['blue_1' 'IMG_20200229_075500.jpg' 0.5090137857900319 (320, 170) 1.0]
['blue

['blue_1' 'IMG_20200229_075502.jpg' 0.5853658536585366 (430, 60) 1.0]
['blue_1' 'IMG_20200229_075502.jpg' 0.5853658536585366 (440, 60) 1.0]
['blue_1' 'IMG_20200229_075502.jpg' 0.5853658536585366 (120, 200) 1.0]
['blue_1' 'IMG_20200229_075502.jpg' 0.5853658536585366 (130, 200) 1.0]
['blue_1' 'IMG_20200229_075502.jpg' 0.5853658536585366 (140, 200) 1.0]
['blue_1' 'IMG_20200229_075502.jpg' 0.5853658536585366 (150, 200) 1.0]
['blue_1' 'IMG_20200229_075502.jpg' 0.5853658536585366 (300, 200) 1.0]
['blue_1' 'IMG_20200229_075502.jpg' 0.5853658536585366 (420, 200) 1.0]
['blue_1' 'IMG_20200229_075502.jpg' 0.5853658536585366 (430, 200) 1.0]
['blue_1' 'IMG_20200229_075502.jpg' 0.5853658536585366 (440, 200) 1.0]
['blue_1' 'IMG_20200229_075502.jpg' 0.5853658536585366 (460, 200) 1.0]
['blue_1' 'IMG_20200229_075502.jpg' 0.5853658536585366 (470, 200) 1.0]
['blue_1' 'IMG_20200229_075502.jpg' 0.5853658536585366 (480, 200) 1.0]
['blue_1' 'IMG_20200229_075502.jpg' 0.5853658536585366 (420, 210) 1.0]
['blue_1

['blue_1' 'IMG_20200229_075504.jpg' 0.5853658536585366 (410, 60) 1.0]
['blue_1' 'IMG_20200229_075504.jpg' 0.5853658536585366 (420, 60) 1.0]
['blue_1' 'IMG_20200229_075504.jpg' 0.5853658536585366 (430, 60) 1.0]
['blue_1' 'IMG_20200229_075504.jpg' 0.5853658536585366 (440, 60) 1.0]
['blue_1' 'IMG_20200229_075504.jpg' 0.5853658536585366 (460, 60) 1.0]
['blue_1' 'IMG_20200229_075504.jpg' 0.5853658536585366 (110, 200) 1.0]
['blue_1' 'IMG_20200229_075504.jpg' 0.5853658536585366 (120, 200) 1.0]
['blue_1' 'IMG_20200229_075504.jpg' 0.5853658536585366 (270, 200) 1.0]
['blue_1' 'IMG_20200229_075504.jpg' 0.5853658536585366 (280, 200) 1.0]
['blue_1' 'IMG_20200229_075504.jpg' 0.5853658536585366 (290, 200) 1.0]
['blue_1' 'IMG_20200229_075504.jpg' 0.5853658536585366 (410, 200) 1.0]
['blue_1' 'IMG_20200229_075504.jpg' 0.5853658536585366 (420, 200) 1.0]
['blue_1' 'IMG_20200229_075504.jpg' 0.5853658536585366 (460, 200) 1.0]
['blue_1' 'IMG_20200229_075504.jpg' 0.5853658536585366 (470, 200) 1.0]
['blue_1' '

['blue_1' 'IMG_20200229_075505.jpg' 0.5853658536585366 (420, 60) 1.0]
['blue_1' 'IMG_20200229_075505.jpg' 0.5853658536585366 (440, 60) 1.0]
['blue_1' 'IMG_20200229_075505.jpg' 0.5853658536585366 (60, 180) 1.0]
['blue_1' 'IMG_20200229_075505.jpg' 0.5853658536585366 (100, 180) 1.0]
['blue_1' 'IMG_20200229_075505.jpg' 0.5853658536585366 (320, 190) 1.0]
['blue_1' 'IMG_20200229_075505.jpg' 0.5853658536585366 (370, 190) 1.0]
['blue_1' 'IMG_20200229_075505.jpg' 0.5853658536585366 (380, 190) 1.0]
['blue_1' 'IMG_20200229_075505.jpg' 0.5853658536585366 (390, 190) 1.0]
['blue_1' 'IMG_20200229_075505.jpg' 0.5853658536585366 (400, 190) 1.0]
['blue_1' 'IMG_20200229_075505.jpg' 0.5853658536585366 (460, 190) 1.0]
['blue_1' 'IMG_20200229_075505.jpg' 0.5853658536585366 (480, 190) 1.0]
['blue_1' 'IMG_20200229_075505.jpg' 0.5853658536585366 (250, 200) 1.0]
['blue_1' 'IMG_20200229_075505.jpg' 0.5853658536585366 (260, 200) 1.0]
['blue_1' 'IMG_20200229_075505.jpg' 0.5853658536585366 (270, 200) 1.0]
['blue_1'

['blue_1' 'IMG_20200229_075507.jpg' 0.5853658536585366 (0, 190) 1.0]
['blue_1' 'IMG_20200229_075507.jpg' 0.5853658536585366 (10, 190) 1.0]
['blue_1' 'IMG_20200229_075507.jpg' 0.5853658536585366 (40, 190) 1.0]
['blue_1' 'IMG_20200229_075507.jpg' 0.5853658536585366 (50, 190) 1.0]
['blue_1' 'IMG_20200229_075507.jpg' 0.5853658536585366 (340, 200) 1.0]
['blue_1' 'IMG_20200229_075507.jpg' 0.5853658536585366 (350, 200) 1.0]
['blue_1' 'IMG_20200229_075507.jpg' 0.5853658536585366 (450, 200) 1.0]
['blue_1' 'IMG_20200229_075507.jpg' 0.5853658536585366 (50, 210) 1.0]
['blue_1' 'IMG_20200229_075507.jpg' 0.5853658536585366 (210, 210) 1.0]
['blue_1' 'IMG_20200229_075507.jpg' 0.5853658536585366 (220, 210) 1.0]
['blue_1' 'IMG_20200229_075507.jpg' 0.5853658536585366 (350, 210) 1.0]
['blue_1' 'IMG_20200229_075507.jpg' 0.5853658536585366 (360, 210) 1.0]
['blue_1' 'IMG_20200229_075507.jpg' 0.5853658536585366 (370, 210) 1.0]
['blue_1' 'IMG_20200229_075507.jpg' 0.5853658536585366 (400, 210) 1.0]
['blue_1' 'I

In [7]:
# window only - 1.36s
# features only - 1.47s
# classify per image - 2.82s
# classify per scale - 8.47s
# dsiplay, features only - 52.64s